<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mMakeHdr

Much of the Montage processing is based on the specification of an output image as captured in an ASCII file version of the output FITS header.  Sometimes such a header is cloned from an existing FITS file and sometimes created from scratch using an editor or the utility mHdr.  

Another common approach is the create a header is to base it on a table of source locations or image metadata by drawing a bounding box around the table contents.  mMakeHdr reads through a table and determines such a bounding box using a spherical geometry variant of the rotating calipers technique from computational geometry.  The box can either be of overall minimum size or can include the constraint that North be oriented upward in the image.

In [5]:
from MontagePy.main import mMakeHdr, mViewer

help(mMakeHdr)

Help on built-in function mMakeHdr in module MontagePy.main:

mMakeHdr(...)
    mMakeHdr subtracts one image from another (both already in the same projection).
    
    Parameters
    ----------
    tblfile : str
        Input image metadata table.
    template : str
        Output image header template.
    csys : str, optional
        Coordinate system (e.g. 'EquJ', 'Galactic').  Fairly forgiving.
    equinox : float, optional
        Coordinate system equinox (e.g. 2000.0).
    pixelScale : float, optional
        Pixel scale in degrees. If zero the scale will be deduced from the image metadata table.
    northAligned : int, optional
        Defaults to minimum bounding box around input images.  This forces template to be north-aligned.
    pad : float, optional
        Optional extra padding around output template.
    isPercentage : bool, optional
        Pad is in pixels by default.  This changes that to a percentage of the image size.
    maxPixel : int, optional
        Settin

## mMakeHdr Example

The principle parameters for mMakeHdr are the table file to be fit (either just coordinates or image metadata) and the output header file name.  There are optional settings for specifying the coordinate system to use, the pixel scale (the default is derived from the table), and whether the image should be "North-up".

We have a set of images retrieved from the 2MASS archive that overlap with a 1-degree box around M17.  Using the metadata from those image and mMakeHdr, we will derive a header file which is somewhat larger, since it will bound the entirity of those images:


In [7]:
rtn = mMakeHdr('M17/remote.tbl', 'work/M17/bounding.hdr')

print(rtn)

{'status': '0', 'count': 49, 'ncube': 0, 'naxis1': 4415, 'naxis2': 4909, 'clon': 275.18944550132534, 'clat': -16.247785977590365, 'lonsize': 1.2263986999999998, 'latsize': 1.36362202, 'posang': 359.9540632922402, 'lon1': 275.82978058308714, 'lat1': -16.929059333050994, 'lon2': 274.54797105435335, 'lat2': -16.928072711843267, 'lon3': 274.55353923516526, 'lat3': -15.564603608451247, 'lon4': 275.82648962400526, 'lat4': -15.565583410970302}


### Header

Here is the generated header:

In [8]:
with open('work/M17/bounding.hdr', 'r') as fin:
    print(fin.read(), end='')

SIMPLE  = T
BITPIX  = -64
NAXIS   = 2
NAXIS1  = 4415
NAXIS2  = 4909
CTYPE1  = 'RA---TAN'
CTYPE2  = 'DEC--TAN'
EQUINOX = 2000
CRVAL1  =  275.189445501
CRVAL2  =  -16.247785978
CRPIX1  =      2208.0000
CRPIX2  =      2455.0000
CDELT1  =   -0.000277780
CDELT2  =    0.000277780
CROTA2  =  359.954063292
END


## Error Handling

If mMakeHdr encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user asks for a non-existent input table:

In [4]:
rtn = mMakeHdr('M17/unknown.tbl', "work/M17/bounding.hdr")

print(rtn)

{'status': '1', 'msg': b'Invalid table file: M17/unknown.tbl'}


&nbsp;

## Classic Montage: mMakeHdr as a Stand-Alone Program

### mMakeHdr Unix/Windows Command-line Arguments

<p>mMakeHdr can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mMakeHdr [-d level] [-s statusfile] [-p(ixel-scale) cdelt | -P maxpixel] [-e edgepixels] [-n] images.tbl template.hdr [system [equinox]] (where system = EQUJ|EQUB|ECLJ|ECLB|GAL|SGAL)
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mMakeHdr can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mMakeHdr                                                             */
/*                                                                       */
/*  Create the best header 'bounding' a table or set of tables (each     */
/*  with image metadata or point sources).                               */
/*                                                                       */
/*   char  *tblfile        Input image metadata table or source table    */
/*                         or table of tables                            */
/*                                                                       */
/*   char  *template       Output image header template                  */
/*                                                                       */
/*   char  *csys           Coordinate system (e.g. 'EquJ', 'Galactic').  */
/*                         Fairly forgiving                              */
/*                                                                       */
/*   double equinox        Coordinate system equinox (e.g. 2000.0)       */
/*                                                                       */
/*   double pixelScale     Pixel scale in degrees                        */
/*                                                                       */
/*   int    northAligned   Defaults to minimum bounding box around       */
/*                         input images.  This forces template to be     */
/*                         north-aligned                                 */
/*                                                                       */
/*   double pad            Optional extra padding around output template */
/*                                                                       */
/*   int    isPercentage   Pad is in pixels by default.  This changes    */
/*                         that to a percentage of the image size        */
/*                                                                       */
/*   int    maxPixel       Setting the pixel scale can result in really  */
/*                         big images.  This forces a maximum number     */
/*                         of pixels in NAXIS1, NAXIS2                   */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mMakeHdrReturn *mMakeHdr(char *tblfile, char *template, char *csysin, double equinox, double pixelScale, 
                                int northAligned, double pad, int isPercentage, int maxPixel, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mMakeHdrReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   char   note[1024];    // Cautionary message (only there if needed).   
   int    count;         // Number of images in metadata table.
   int    ncube;         // Number of images that have 3/4 dimensions.
   int    naxis1;        // X axis pixel count in output template.
   int    naxis2;        // Y axis pixel count in output template.
   double clon;          // Center longitude for template.
   double clat;          // Center latitude for template.
   double lonsize;       // Template dimensions in X.
   double latsize;       // Template dimensions in Y.
   double posang;        // Rotation angle of template.
   double lon1;          // Image corners (lon of first corner).
   double lat1;          // Image corners (lat of first corner).
   double lon2;          // Image corners (lon of second corner).
   double lat2;          // Image corners (lat of second corner).
   double lon3;          // Image corners (lon of third corner).
   double lat3;          // Image corners (lat of third corner).
   double lon4;          // Image corners (lon of fourth corner).
   double lat4;          // Image corners (lat of fourth corner).
};
</pre>